In [1]:
GLUE_TASKS = ["cola", "mnli", "mnli-mm", "mrpc", "qnli", "qqp", "rte", "sst2", "stsb", "wnli"]

In [2]:
task = "sst2"
model_checkpoint = "bert-base-multilingual-uncased"
batch_size = 16

In [3]:
import transformers
import torch
from torch import nn
print(transformers.__version__)

4.12.5


In [4]:
import wandb
#0a147497e5856fdc2538a1adbb9935ad0857277b
wandb.init()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 914, in init
    wi.setup(kwargs)
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_init.py", line 185, in setup
    _silent=(settings._quiet or settings._silent) is True,
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_login.py", line 274, in _login
    wlogin.prompt_api_key()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_login.py", line 202, in prompt_api_key
    key, status = self._prompt_api_key()
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/wandb_login.py", line 186, in _prompt_api_key
    no_create=self._settings.force if self._settings else None,
  File "/opt/conda/lib/python3.7/site-packages/wandb/sdk/lib/apikey.py", line 126, in prompt_api_key
    key = input_callback(api_ask).strip()
  File "/opt/conda/lib/python3.7/site-package

Exception: problem

随机显示

In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

加载分词器

In [ ]:
task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
}

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

def preprocess_function(examples):
    if sentence2_key is None:
        return tokenizer(examples[sentence1_key],max_length = 100,pad_to_max_length = True,)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], max_length = 100,pad_to_max_length = True, )

# Loading the dataset

In [ ]:
import pandas as pd
# Read data from files 
En_data = pd.read_csv( "../input/2021-semeval-task6/train.En.csv", header=0)
print(En_data.shape)
# Read data from files 
Ar_data = pd.read_csv( "../input/2021-semeval-task6/train.Ar.csv", header=0)
print(Ar_data.shape)

处理文本

In [ ]:
import re
def clearTextFunc(text):
    clear_text = " ".join(text.split())   # 去除回车和换行符
    clear_text = re.sub(r'http://[a-zA-Z0-9.?/&=:]*'," ",  clear_text)   # 去除URL
    clear_text = re.sub(r'https://[a-zA-Z0-9.?/&=:]*'," ",  clear_text)   # 去除URL
    clear_text = re.sub(r'@[a-zA-Z0-9.?/&=:]*'," ",  clear_text)   # 去除@
    clear_text = re.sub(r'/s'," ",  clear_text)   # 去除/s
    clear_text = re.sub(r'/j'," ",  clear_text)   # 去除/j

    clear_text  = clear_text.lower()
    
    return(clear_text)

In [ ]:
def clearTexts(data,pos='tweet'):
    # Get the number of Texts based on the dataframe column size
    global num_Texts
    num_Texts = data[pos].size
    # Initialize an empty list to hold the clean Texts
    print("Cleaning and parsing the training set Texts...\n")
    # Loop over each Text; create an index i that goes from 0 to the length# of the Text list 
    for i in range( 0, num_Texts ):
        # If the index is evenly divisible by 1000, print a message
        if( (i+1)%1000 == 0 ):
            print("Text %d of %d\n" % ( i+1, num_Texts ))   
        # Call our function for each one, and add the result to the list of    # clean Texts
        if(str(data[pos][i])!="nan"):
            clean_train_Texts.append( clearTextFunc( data[pos][i] ) )
        else:
            clean_train_Texts.append( clearTextFunc("" ) )

In [ ]:
clean_train_Texts = []
num_Texts = 0
clearTexts(En_data)
clean_train_Texts_En = clean_train_Texts
num_Texts_En = num_Texts

clean_train_Texts = []
num_Texts = 0
clearTexts(Ar_data)
clean_train_Texts_Ar = clean_train_Texts
num_Texts_Ar = num_Texts

# Task A

## 处理标签

In [ ]:
import numpy as np
import torch

label_En = En_data['sarcastic']
label_En=np.array(label_En)
label_En = label_En.reshape(num_Texts_En,1)
print(label_En.shape)


label_Ar = Ar_data['sarcastic']
label_Ar=np.array(label_Ar)
label_Ar = label_Ar.reshape(num_Texts_Ar,1)
print(label_Ar.shape)

## 划分数据集

In [ ]:
En=clean_train_Texts_En[0:1300]
clean_train_Texts = []
num_Texts = 0
clearTexts(Ar_data,pos='tweet')
a = clean_train_Texts

l_En=label_En[0:1300]
for i in range(200,501):
    En.append(str(a[i]))
    l_En=np.append(l_En,[0])


Ar=clean_train_Texts_Ar[0:800]
l_Ar=label_Ar[0:800]
clean_train_Texts = []
num_Texts = 0
clearTexts(Ar_data,pos='tweet')
a = clean_train_Texts
for i in range(100,301):
    Ar.append(a[i])
    l_Ar=np.append(l_Ar,[0])

In [ ]:
l_En=l_En.reshape(len(l_En),1)
l_Ar=l_Ar.reshape(len(l_Ar),1)

In [ ]:
from sklearn.model_selection import train_test_split
train_En_X,test_En_X, train_En_y, test_En_y = train_test_split(En, l_En, test_size = 0.1, random_state = 0)
train_Ar_X,test_Ar_X, train_Ar_y, test_Ar_y = train_test_split(Ar, l_Ar, test_size = 0.1, random_state = 0)

In [ ]:
from datasets import Dataset,DatasetDict
dataset_En = DatasetDict({'train':Dataset.from_dict({"sentence": train_En_X,'label':train_En_y}), 'test':Dataset.from_dict({"sentence": test_En_X,'label':test_En_y})})
dataset_Ar = DatasetDict({'train':Dataset.from_dict({"sentence": train_Ar_X,'label':train_Ar_y}),'test':Dataset.from_dict({"sentence": test_Ar_X,'label':test_Ar_y})})

In [ ]:
dataset_En

## 分词

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
encoded_dataset_En = dataset_En.map(preprocess_function, batched=True)
encoded_dataset_Ar = dataset_Ar.map(preprocess_function, batched=True)

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model1 = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

训练参数

In [ ]:
metric_name = "f1-score"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    save_total_limit = 3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
#     push_to_hub=True,
)

评估函数

In [ ]:
from sklearn.metrics import f1_score,accuracy_score
def compute_metrics1(eval_pred):
    y_pred, y_true = eval_pred
    y_pred = np.argmax(y_pred, axis=1)
    F1 = f1_score(y_true, y_pred,labels=[0,1])
    acc = accuracy_score(y_true, y_pred)
    return{'f1-score': F1,'accuracy': acc}

训练器

In [ ]:
trainer = Trainer(
    model1,
    args,
    train_dataset=encoded_dataset_En["train"],
    eval_dataset=encoded_dataset_En["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics1
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer = Trainer(
    model1,
    args,
    train_dataset=encoded_dataset_Ar["train"],
    eval_dataset=encoded_dataset_Ar["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics1
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
torch.save(model1,"mlpl1.pkl")

# Task B

## 处理标签

In [ ]:
import numpy as np
import torch
x=[0,1,2,3]
label2 = En_data.copy()
label2.drop(label2.columns[x], axis=1, inplace=True)
label2=np.array(label2)
#nan->0
for i in range(0,num_Texts_En):
    for j in range(0,6):
        if(str(label2[i][j])=="nan"):
            label2[i][j]=0
label2 = label2.reshape(num_Texts_En,6)
print(label2.shape)

In [ ]:
label2.dtype

In [ ]:
l1=l2=l3=l4=l5=l6=0
for i in range(label2.shape[0]):
    l1 += label2[i][0]
    l2 += label2[i][1]
    l3 += label2[i][2]
    l4 += label2[i][3]
    l5 += label2[i][4]
    l6 += label2[i][5]

In [ ]:
print(l1)
print(l2)
print(l3)
print(l4)
print(l5)
print(l6)

## 划分数据集

In [ ]:
from sklearn.model_selection import train_test_split
train2_X,test2_X, train2_y, test2_y = train_test_split(clean_train_Texts_En[0:900], label2[0:900], test_size = 0.1, random_state = 0)
from datasets import Dataset,DatasetDict
dataset2 = DatasetDict({'train':Dataset.from_dict({"sentence": train2_X,'labels':train2_y}),'test':Dataset.from_dict({"sentence": test2_X,'labels':test2_y})})

## 分词

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
encoded_dataset2 = dataset2.map(preprocess_function, batched=True)

## Fine-tuning the model

### 拆分二分类器版本

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model_1 = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model_2 = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model_3 = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model_4 = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model_5 = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model_6 = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
model_7 = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)


收集标签并划分数据集

In [ ]:
t_1=[]
l_1=[]
t_2=[]
l_2=[]
t_3=[]
l_3=[]
t_4=[]
l_4=[]
t_5=[]
l_5=[]
t_6=[]
l_6=[]
for i in range(label2.shape[0]):
    if(label2[i][0]==1):
        t_1.append(clean_train_Texts_En[i])
        l_1.append(1)
    elif(l1!=0):
        l1-=1
        t_1.append(clean_train_Texts_En[i])
        l_1.append(0)
    if(label2[i][1]==1):
        t_2.append(clean_train_Texts_En[i])
        l_2.append(1)
    elif(l2!=0):
        l2-=1
        t_2.append(clean_train_Texts_En[i])
        l_2.append(0)
    if(label2[i][2]==1):
        t_3.append(clean_train_Texts_En[i])
        l_3.append(1)
    elif(l3!=0):
        l3-=1
        t_3.append(clean_train_Texts_En[i])
        l_3.append(0)
    if(label2[i][3]==1):
        t_4.append(clean_train_Texts_En[i])
        l_4.append(1)
    elif(l4!=0):
        l4-=1
        t_4.append(clean_train_Texts_En[i])
        l_4.append(0)
    if(label2[i][4]==1):
        t_5.append(clean_train_Texts_En[i])
        l_5.append(1)
    elif(l5!=0):
        l5-=1
        t_5.append(clean_train_Texts_En[i])
        l_5.append(0)
    if(label2[i][5]==1):
        t_6.append(clean_train_Texts_En[i])
        l_6.append(1)
    elif(l6!=0):
        l6-=1
        t_6.append(clean_train_Texts_En[i])
        l_6.append(0)
        

In [ ]:
l_1=np.array(l_1).reshape(len(l_1),1)
l_2=np.array(l_2).reshape(len(l_2),1)
l_3=np.array(l_3).reshape(len(l_3),1)
l_4=np.array(l_4).reshape(len(l_4),1)
l_5=np.array(l_5).reshape(len(l_5),1)
l_6=np.array(l_6).reshape(len(l_6),1)

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset,DatasetDict
train2_X,test2_X, train2_y, test2_y = train_test_split(t_1, l_1, test_size = 0.2, random_state = 0)
dataset2_l1 = DatasetDict({'train':Dataset.from_dict({"sentence": train2_X,'labels':train2_y}),'test':Dataset.from_dict({"sentence": test2_X,'labels':test2_y})})
train2_X,test2_X, train2_y, test2_y = train_test_split(t_2, l_2, test_size = 0.2, random_state = 0)
dataset2_l2 = DatasetDict({'train':Dataset.from_dict({"sentence": train2_X,'labels':train2_y}),'test':Dataset.from_dict({"sentence": test2_X,'labels':test2_y})})
train2_X,test2_X, train2_y, test2_y = train_test_split(t_3, l_3, test_size = 0.2, random_state = 0)
dataset2_l3 = DatasetDict({'train':Dataset.from_dict({"sentence": train2_X,'labels':train2_y}),'test':Dataset.from_dict({"sentence": test2_X,'labels':test2_y})})
train2_X,test2_X, train2_y, test2_y = train_test_split(t_4, l_4, test_size = 0.2, random_state = 0)
dataset2_l4 = DatasetDict({'train':Dataset.from_dict({"sentence": train2_X,'labels':train2_y}),'test':Dataset.from_dict({"sentence": test2_X,'labels':test2_y})})
train2_X,test2_X, train2_y, test2_y = train_test_split(t_5, l_5, test_size = 0.2, random_state = 0)
dataset2_l5 = DatasetDict({'train':Dataset.from_dict({"sentence": train2_X,'labels':train2_y}),'test':Dataset.from_dict({"sentence": test2_X,'labels':test2_y})})
train2_X,test2_X, train2_y, test2_y = train_test_split(t_6, l_6, test_size = 0.2, random_state = 0)
dataset2_l6 = DatasetDict({'train':Dataset.from_dict({"sentence": train2_X,'labels':train2_y}),'test':Dataset.from_dict({"sentence": test2_X,'labels':test2_y})})

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
encoded_dataset2_l1 = dataset2_l1.map(preprocess_function, batched=True)
encoded_dataset2_l2 = dataset2_l2.map(preprocess_function, batched=True)
encoded_dataset2_l3 = dataset2_l3.map(preprocess_function, batched=True)
encoded_dataset2_l4 = dataset2_l4.map(preprocess_function, batched=True)
encoded_dataset2_l5 = dataset2_l5.map(preprocess_function, batched=True)
encoded_dataset2_l6 = dataset2_l6.map(preprocess_function, batched=True)

In [ ]:
from transformers import TrainingArguments,Trainer
metric_name = "f1-score"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    save_total_limit = 1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
#     push_to_hub=True,
)

from sklearn.metrics import f1_score,accuracy_score
def compute_metrics2(eval_pred):
    y_pred, y_true = eval_pred
    y_pred = np.argmax(y_pred, axis=1)
    F1 = f1_score(y_true, y_pred,labels=[0,1])
    acc = accuracy_score(y_true, y_pred)
    return{'f1-score': F1,'accuracy': acc}


trainer1 = Trainer(
    model_1,
    args,
    train_dataset=encoded_dataset2_l1["train"],
    eval_dataset=encoded_dataset2_l1["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2
)
trainer2 = Trainer(
    model_2,
    args,
    train_dataset=encoded_dataset2_l2["train"],
    eval_dataset=encoded_dataset2_l2["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2
)
trainer3 = Trainer(
    model_3,
    args,
    train_dataset=encoded_dataset2_l3["train"],
    eval_dataset=encoded_dataset2_l3["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2
)
trainer4 = Trainer(
    model_4,
    args,
    train_dataset=encoded_dataset2_l4["train"],
    eval_dataset=encoded_dataset2_l4["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2
)
trainer5 = Trainer(
    model_5,
    args,
    train_dataset=encoded_dataset2_l5["train"],
    eval_dataset=encoded_dataset2_l5["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2
)
trainer6 = Trainer(
    model_6,
    args,
    train_dataset=encoded_dataset2_l6["train"],
    eval_dataset=encoded_dataset2_l6["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2
)

In [ ]:
trainer1.train()
trainer1.evaluate()
model_1.cpu()
trainer2.train()
model_2.cpu()
trainer3.train()
model_3.cpu()
trainer4.train()
model_4.cpu()
trainer5.train()
model_5.cpu()
trainer6.train()
model_6.cpu()

### 更换损失函数版本

计算权重

In [ ]:
# 统计各类标签数量，用总数/当前类数，以增大比重，也可以用其他方法计算权重
l1=l2=l3=l4=l5=l6=0
total_num=len(dataset2['train']['labels'])
for i in range(total_num):
    l1 += dataset2['train']['labels'][i][0]
    l2 += dataset2['train']['labels'][i][1]
    l3 += dataset2['train']['labels'][i][2]
    l4 += dataset2['train']['labels'][i][3]
    l5 += dataset2['train']['labels'][i][4]
    l6 += dataset2['train']['labels'][i][5]

print(l1)
print(l2)
print(l3)
print(l4)
print(l5)
print(l6)
label_weight = torch.FloatTensor([total_num/l1, total_num/l2, total_num/l3, total_num/l4, total_num/l5, total_num/l6]).cuda()

In [ ]:
label_weight

DiceLoss损失函数实现

In [ ]:
import torch
import torch.nn as nn
# 针对二分类任务的 Dice Loss
class DiceLoss(nn.Module):
    def __init__(self, size_average=True):
        super(DiceLoss, self).__init__()
        self.size_average = size_average
    def forward(self, input, targets):
        # 如果模型最后没有 nn.Sigmoid()，那么这里就需要对预测结果计算一次 Sigmoid 操作
        input = nn.Sigmoid()(input)
        
        # 获取每个批次的大小 N
        N = targets.size()[0]
        # 平滑变量
        smooth = 1
        # 将宽高 reshape 到同一纬度
        input_flat = input.view(N, -1)
        targets_flat = targets.view(N, -1)
    
        # 计算交集
        intersection = input_flat * targets_flat 
        N_dice_eff = (2 * intersection.sum(1) + smooth) / (input_flat.sum(1) + targets_flat.sum(1) + smooth)
        # 计算一个批次中平均每张图的损失
        loss = 1 - N_dice_eff.sum() / N
        
         # Loss Function的常规操作，mean 与 sum 的区别不大，相当于学习率设置不一样而已
        if self.size_average:
            loss = loss.mean()
        else:
            loss = loss.sum()
        return loss
    
# 针对 Multi-Label 任务的 Dice Loss
class DiceLoss_MultiLabel(nn.Module):
    def __init__(self,size_average=True):
        super(DiceLoss_MultiLabel, self).__init__()
        self.size_average = size_average

    def forward(self, pred, target):
        criterion = DiceLoss(self.size_average)
        loss = torch.zeros(1,target.shape[1]).cuda()

        # 对每个 Label 计算一次 Dice Loss
        for label in range(target.shape[1]):
            batch_loss = criterion(pred[:,label],target[:,label])
            loss[0,label] = batch_loss.mean()

        # Loss Function的常规操作
        if self.size_average:
            loss = loss.mean()
        else:
            loss = loss.sum()

        return loss

FocalLoss损失函数实现

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
# 针对二分类任务的 Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2, size_average=True):
        super(FocalLoss, self).__init__()
        self.alpha = torch.tensor(alpha).cuda()
        self.gamma = gamma
        self.size_average = size_average

    def forward(self, pred, target):
        # 如果模型最后没有 nn.Sigmoid()，那么这里就需要对预测结果计算一次 Sigmoid 操作
        pred = nn.Sigmoid()(pred)

        # 展开 pred 和 target,此时 pred.size = target.size = (BatchSize,1) 
        pred = pred.view(-1,1)
        target = target.view(-1,1)

        # 此处将预测样本为正负的概率都计算出来，此时 pred.size = (BatchSize,2)
        pred = torch.cat((1-pred,pred),dim=1)

        # 根据 target 生成 mask，即根据 ground truth 选择所需概率
        # 用大白话讲就是：
        # 当标签为 1 时，我们就将模型预测该样本为正类的概率代入公式中进行计算
        # 当标签为 0 时，我们就将模型预测该样本为负类的概率代入公式中进行计算
        class_mask = torch.zeros(pred.shape[0],pred.shape[1]).cuda()
        # 这里的 scatter_ 操作不常用，其函数原型为:
        # scatter_(dim,index,src)->Tensor
        # Writes all values from the tensor src into self at the indices specified in the index tensor. 
        # For each value in src, its output index is specified by its index in src for dimension != dim and by the corresponding value in index for dimension = dim.
        class_mask.scatter_(1, target.view(-1, 1).long(), 1.)

        # 利用 mask 将所需概率值挑选出来
        probs = (pred * class_mask).sum(dim=1).view(-1,1)
        probs = probs.clamp(min=0.0001,max=1.0)

        # 计算概率的 log 值
        log_p = probs.log()

        # 根据论文中所述，对 alpha　进行设置（该参数用于调整正负样本数量不均衡带来的问题）
        alpha = torch.ones(pred.shape[0],pred.shape[1]).cuda()
        alpha[:,0] = alpha[:,0] * (1-self.alpha)
        alpha[:,1] = alpha[:,1] * self.alpha
        alpha = (alpha * class_mask).sum(dim=1).view(-1,1)

        # 根据 Focal Loss 的公式计算 Loss
        batch_loss = -alpha*(torch.pow((1-probs), self.gamma))*log_p

         # Loss Function的常规操作，mean 与 sum 的区别不大，相当于学习率设置不一样而已
        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()

        return loss

# 针对 Multi-Label 任务的 Focal Loss
class FocalLoss_MultiLabel(nn.Module):
    def __init__(self, alpha=0.25, gamma=2, size_average=True):
        super(FocalLoss_MultiLabel, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.size_average = size_average

    def forward(self, pred, target):
        criterion = FocalLoss(self.alpha,self.gamma,self.size_average)
        loss = torch.zeros(1,target.shape[1]).cuda()

        # 对每个 Label 计算一次 Focal Loss
        for label in range(target.shape[1]):
            batch_loss = criterion(pred[:,label],target[:,label])
            loss[0,label] = batch_loss.mean()

        # Loss Function的常规操作
        if self.size_average:
            loss = loss.mean()
        else:
            loss = loss.sum()

        return loss

In [ ]:
from transformers import DistilBertModel, BertModel
from torch.nn import MSELoss,CrossEntropyLoss,BCEWithLogitsLoss
class DistilBERTClass(torch.nn.Module):
    def __init__(self,num_labels=6):
        super(DistilBERTClass, self).__init__()
        self.bert = BertModel.from_pretrained(model_checkpoint)
        self.config=self.bert.config
        self.num_labels=num_labels
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, num_labels)
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        pooled_output = outputs[1]
        
        pooled_output = self.pre_classifier(pooled_output)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        
        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                if self.num_labels == 1:
                    loss = loss_fct(logits.squeeze(), labels.squeeze()) 
                else:
                    loss = loss_fct(logits, labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = FocalLoss_MultiLabel() #DiceLoss_MultiLabel()#BCEWithLogitsLoss(weight=label_weight,reduction='mean') #BCEWithLogitsLoss()      
                loss = loss_fct(logits, labels)
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output
#         print(self.config.problem_type)
        output = (logits,) + outputs[2:]
        return ((loss,) + output) if loss is not None else output
        
        
#         output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask)
#         hidden_state = output_1[0]
#         pooler = hidden_state[:, 0]
#         pooler = self.pre_classifier(pooler)
#         pooler = torch.nn.ReLU()(pooler)
#         pooler = self.dropout(pooler)
#         output = self.classifier(pooler)
#         return output

model2=DistilBERTClass()

训练参数

In [ ]:
from transformers import TrainingArguments,Trainer
metric_name = "macro-f1 score"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    save_total_limit = 3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
#     push_to_hub=True,
)

评估函数

In [ ]:
m = nn.Sigmoid()
from sklearn.metrics import f1_score,accuracy_score
def compute_metrics2(eval_pred):
    y_pred, y_true = eval_pred
    y_pred = m(torch.tensor(y_pred))
    y_pred = np.int64(y_pred>=0.5)
#     y_pred = np.array(y_pred) >=0.5
#     y_pred = list(map(list, zip(*y_pred)))      #转置
#     y_true = list(map(list, zip(*y_true)))      #转置
    ans = f1_score(y_true, y_pred,average='macro')
    acc = accuracy_score(y_true, y_pred)
    return{'macro-f1 score':ans,'accuracy': acc}

训练器

In [ ]:
encoded_dataset2["train"]

In [ ]:
trainer = Trainer(
    model2,
    args,
    train_dataset=encoded_dataset2["train"],
    eval_dataset=encoded_dataset2["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics2
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

保存模型

In [ ]:
torch.save(model2,"mlpl2.pkl")


# Task C

In [ ]:
task = "mrpc"

## 处理数据

866个英文有重述版本

500个阿拉伯语有重述版本

分开训练，不然会有问题

清理文本

In [ ]:
def clearTexts2(data):
    # Get the number of Texts based on the dataframe column size
    global num_Texts
    # Initialize an empty list to hold the clean Texts
    print("Cleaning and parsing the training set Texts...\n")
    # Loop over each Text; create an index i that goes from 0 to the length# of the Text list 
    for i in range( 0, num_Texts ):
        # If the index is evenly divisible by 300, print a message
        if( (i+1)%300 == 0 ):
            print("Text %d of %d\n" % ( i+1, num_Texts ))   
        # Call our function for each one, and add the result to the list of    # clean Texts
        clean_tweet_Texts.append( clearTextFunc( data["tweet"][i] ))
        clean_rephrase_Texts.append( clearTextFunc( data["rephrase"][i] ))

In [ ]:
clean_tweet_Texts=[]
clean_rephrase_Texts=[]
num_Texts=866
clearTexts2(En_data)
clean_tweet_Texts_En=clean_tweet_Texts
clean_rephrase_Texts_En=clean_rephrase_Texts


clean_tweet_Texts=[]
clean_rephrase_Texts=[]
num_Texts=500
clearTexts2(Ar_data)
clean_tweet_Texts_Ar=clean_tweet_Texts
clean_rephrase_Texts_Ar=clean_rephrase_Texts

In [ ]:
train_En=[]
for i in range(866):
    train_En.append([clean_rephrase_Texts_En[i],clean_tweet_Texts_En[i]])
    train_En.append([clean_tweet_Texts_En[i],clean_rephrase_Texts_En[i]])

train_Ar=[]
for i in range(500):
    train_Ar.append([clean_rephrase_Texts_Ar[i],clean_tweet_Texts_Ar[i]])
    train_Ar.append([clean_tweet_Texts_Ar[i],clean_rephrase_Texts_Ar[i]])

处理标签

In [ ]:
import numpy as np
import torch

label_En = []
for i in range(866):
    label_En.append(1)
    label_En.append(0)
label_En=np.array(label_En)
label_En = label_En.reshape(1732,1)
print(label_En.shape)


label_Ar = []
for i in range(500):
    label_Ar.append(1)
    label_Ar.append(0)
    
label_Ar=np.array(label_Ar)
label_Ar = label_Ar.reshape(1000,1)
print(label_Ar.shape)

## 划分数据集

In [ ]:
from sklearn.model_selection import train_test_split
train_En_X,test_En_X, train_En_y, test_En_y = train_test_split(train_En, label_En, test_size = 0.2, random_state = 0)
train_Ar_X,test_Ar_X, train_Ar_y, test_Ar_y = train_test_split(train_Ar, label_Ar, test_size = 0.2, random_state = 0)

In [ ]:
train_En_X_s={'s1':[],'s2':[]}
n=len(train_En_y)
for i in range(n):
    train_En_X_s['s1'].append(train_En_X[i][0])
    train_En_X_s['s2'].append(train_En_X[i][1])
    
train_Ar_X_s={'s1':[],'s2':[]}
n=len(train_Ar_y)
for i in range(n):
    train_Ar_X_s['s1'].append(train_Ar_X[i][0])
    train_Ar_X_s['s2'].append(train_Ar_X[i][1])
    
test_En_X_s={'s1':[],'s2':[]}
n=len(test_En_y)
for i in range(n):
    test_En_X_s['s1'].append(test_En_X[i][0])
    test_En_X_s['s2'].append(test_En_X[i][1])

test_Ar_X_s={'s1':[],'s2':[]}
n=len(test_Ar_y)
for i in range(n):
    test_Ar_X_s['s1'].append(test_Ar_X[i][0])
    test_Ar_X_s['s2'].append(test_Ar_X[i][1])

In [ ]:
from datasets import Dataset,DatasetDict
dataset_En = DatasetDict({'train':Dataset.from_dict({"sentence1": train_En_X_s['s1'], "sentence2": train_En_X_s['s2'], 'label':train_En_y}),'test':Dataset.from_dict({"sentence1": test_En_X_s['s1'], "sentence2": test_En_X_s['s2'], 'label':test_En_y})})
dataset_Ar = DatasetDict({'train':Dataset.from_dict({"sentence1": train_Ar_X_s['s1'], "sentence2": train_Ar_X_s['s2'], 'label':train_Ar_y}),'test':Dataset.from_dict({"sentence1": test_Ar_X_s['s1'], "sentence2": test_Ar_X_s['s2'], 'label':test_Ar_y})})

In [ ]:
dataset_Ar

In [ ]:
show_random_elements(dataset_Ar['train'])

## 分词

In [ ]:
sentence1_key, sentence2_key = task_to_keys[task]
encoded_dataset_En = dataset_En.map(preprocess_function, batched=True)
encoded_dataset_Ar = dataset_Ar.map(preprocess_function, batched=True)

In [ ]:
encoded_dataset_Ar['test']

## Fine-tuning the model

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model3 = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", num_labels=num_labels)

训练参数

In [ ]:
metric_name = "accuracy"
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    save_total_limit = 3,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=8,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
#     push_to_hub=True,
)

评估函数

In [ ]:
from sklearn.metrics import accuracy_score
def compute_metrics3(eval_pred):
    y_pred, y_true = eval_pred
    y_pred = np.argmax(y_pred, axis=1)
    acc = accuracy_score(y_true, y_pred)
    return{'accuracy': acc}

训练器

In [ ]:
trainer = Trainer(
    model3,
    args,
    train_dataset=encoded_dataset_En["train"],
    eval_dataset=encoded_dataset_En["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics3
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer = Trainer(
    model3,
    args,
    train_dataset=encoded_dataset_Ar["train"],
    eval_dataset=encoded_dataset_Ar["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics3
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
torch.save(model3,"mlpl3.pkl")

In [ ]:

# classifier = pipeline('sentiment-analysis',model=model3,tokenizer=tokenizer)

In [ ]:
# result = classifier(test_examples[0]['text'])

# INFER阶段：使用模型进行预测

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, random_split
import torch

## Task A

In [ ]:
import pandas as pd
# Read data from files 
En_test = pd.read_csv( "../input/2021-semeval-task6/taskA.En.input.csv", header=0)
print(En_test.shape)
# Read data from files 
Ar_test = pd.read_csv( "../input/2021-semeval-task6/taskA_AR_test.csv", header=0)
print(Ar_test.shape)

In [ ]:
clean_train_Texts = []
num_Texts = 0
clearTexts(En_test,pos='text')
clean_train_Texts_En=clean_train_Texts

clean_train_Texts = []
num_Texts = 0
clearTexts(Ar_test,pos='text')
clean_train_Texts_Ar=clean_train_Texts

英语部分

In [ ]:
text=Dataset.from_dict({"sentence": clean_train_Texts_En})
encoded_text = text.map(preprocess_function, batched=True)
pred =trainer.predict(encoded_text)
final_preds = np.argmax(pred[0], axis=1).tolist()

In [ ]:
sum(final_preds)

In [ ]:
# Create submission file
submission = pd.DataFrame()
submission['task_a_en'] = final_preds
submission.to_csv('dummy_a_en.txt', index=False)

阿拉伯语部分

In [ ]:
text=Dataset.from_dict({"sentence": clean_train_Texts_En})
encoded_text = text.map(preprocess_function, batched=True)
pred =trainer.predict(encoded_text)
final_preds = np.argmax(pred[0], axis=1).tolist()

In [ ]:
sum(final_preds)

In [ ]:
# Create submission file
submission = pd.DataFrame()
submission['task_a_ar'] = final_preds
submission.to_csv('dummy_a_ar.txt', index=False)

## Task B

In [ ]:
import pandas as pd
# Read data from files 
En_test = pd.read_csv( "../input/2021-semeval-task6/taskB.En.input.csv", header=0)
print(En_test.shape)

In [ ]:
clean_train_Texts = []
num_Texts = 0
clearTexts(En_test,pos='text')
clean_train_Texts_En=clean_train_Texts

# all_input_ids = encode_fn(clean_train_Texts_En)

拆分多二分类器版本

In [ ]:
from transformers import pipeline
test_dataset = {
  'text': clean_train_Texts_En,
}

classfier = pipeline('sentiment-analysis', model=model_1, tokenizer=tokenizer)
result1 = classfier(test_dataset["text"])

classfier = pipeline('sentiment-analysis', model=model_2, tokenizer=tokenizer)
result2 = classfier(test_dataset["text"])

classfier = pipeline('sentiment-analysis', model=model_3, tokenizer=tokenizer)
result3 = classfier(test_dataset["text"])

classfier = pipeline('sentiment-analysis', model=model_4, tokenizer=tokenizer)
result4 = classfier(test_dataset["text"])

classfier = pipeline('sentiment-analysis', model=model_5, tokenizer=tokenizer)
result5 = classfier(test_dataset["text"])


classfier = pipeline('sentiment-analysis', model=model_6, tokenizer=tokenizer)
result6= classfier(test_dataset["text"])


In [ ]:
preds=[]
for i in range(len(clean_train_Texts)):
    tmp=[]
    if result1[i]['label']=='LABEL_1':
        tmp.append(1)
    else:
        tmp.append(0)
    if result2[i]['label']=='LABEL_1':
        tmp.append(1)
    else:
        tmp.append(0)
    if result3[i]['label']=='LABEL_1':
        tmp.append(1)
    else:
        tmp.append(0)
    if result4[i]['label']=='LABEL_1':
        tmp.append(1)
    else:
        tmp.append(0)
    if result5[i]['label']=='LABEL_1':
        tmp.append(1)
    else:
        tmp.append(0)
    if result6[i]['label']=='LABEL_1':
        tmp.append(1)
    else:
        tmp.append(0)
    preds.append(preds)
    

In [ ]:
final_preds = np.concatenate(preds, axis=0)

In [ ]:
final_preds=final_preds.T

In [ ]:
sum(final_preds[4])

In [ ]:
# Create submission file
submission = pd.DataFrame()
submission['sarcasm'] = final_preds[0]
submission['irony'] = final_preds[1]
submission['satire'] = final_preds[2]
submission['understatement'] = final_preds[3]
submission['overstatement'] = final_preds[4]
submission['rhetorical_question'] = final_preds[5]
submission.to_csv('dummy_b.txt', index=False)

合并版本

In [ ]:
model=torch.load("mlpl2.pkl")#导入保存的模型

In [ ]:
text=Dataset.from_dict({"sentence":clean_train_Texts_En})
encoded_text = text.map(preprocess_function, batched=True)
pred =trainer.predict(encoded_text)
final_preds = m(torch.tensor(pred[0]))
final_preds = np.int64(final_preds>=0.5)

In [ ]:
final_preds=final_preds.T

In [ ]:
sum(final_preds[4])

In [ ]:
# Create submission file
submission = pd.DataFrame()
submission['sarcasm'] = final_preds[0]
submission['irony'] = final_preds[1]
submission['satire'] = final_preds[2]
submission['understatement'] = final_preds[3]
submission['overstatement'] = final_preds[4]
submission['rhetorical_question'] = final_preds[5]
submission.to_csv('dummy_b.txt', index=False)

## Task C

In [ ]:
import pandas as pd
# Read data from files 
En_test = pd.read_csv( "../input/2021-semeval-task6/taskC.En.input.csv", header=0)
print(En_test.shape)
# Read data from files 
Ar_test = pd.read_csv( "../input/2021-semeval-task6/taskC_AR_test.csv", header=0)
print(Ar_test.shape)

In [ ]:
model=torch.load("mlpl3.pkl")#导入保存的模型

英语部分

In [ ]:
clean_train_Texts = []
num_Texts = 0
clearTexts(En_test,pos='text_0')
clean_train_Texts_En_0=clean_train_Texts
clean_train_Texts = []
num_Texts = 0
clearTexts(En_test,pos='text_1')
clean_train_Texts_En_1=clean_train_Texts


In [ ]:
text=Dataset.from_dict({"sentence1": clean_train_Texts_En_0,'sentence2':clean_train_Texts_En_1})
encoded_text = text.map(preprocess_function, batched=True)
pred =trainer.predict(encoded_text)
final_preds = np.argmax(pred[0], axis=1).tolist()

In [ ]:
sum(final_preds)

In [ ]:
# Create submission file
submission = pd.DataFrame()
submission['task_c_en'] = final_preds
submission.to_csv('dummy_c_en.txt', index=False)

阿拉伯语部分

In [ ]:
clean_train_Texts = []
num_Texts = 0
clearTexts(Ar_test,pos='text_0')
clean_train_Texts_Ar_0=clean_train_Texts
clean_train_Texts = []
num_Texts = 0
clearTexts(Ar_test,pos='text_1')
clean_train_Texts_Ar_1=clean_train_Texts


In [ ]:
text=Dataset.from_dict({"sentence1": clean_train_Texts_En_0,'sentence2':clean_train_Texts_En_1})
encoded_text = text.map(preprocess_function, batched=True)
pred =trainer.predict(encoded_text)
final_preds = np.argmax(pred[0], axis=1).tolist()

In [ ]:
sum(final_preds)

In [ ]:
# Create submission file
submission = pd.DataFrame()
submission['task_c_ar'] = final_preds
submission.to_csv('dummy_c_ar.txt', index=False)